# Week 3 Lab

## Part 1: Getting the dataframe

In [1]:
# imports
import bs4
from bs4 import BeautifulSoup
import requests


In [2]:
# get the soup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
response
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
#make a dataframe
myList=[]
for a in soup.table.tbody.children:
    if type(a)==bs4.element.Tag:
        temp=[]
        for b in a.children:
            if type(b)==bs4.element.Tag and b.get_text()!='':
                temp.append(b.get_text().strip())        
        myList.append(temp[:])

import pandas as pd
df=pd.DataFrame(myList[1:])
df.columns=myList[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# Remove "Not assigned"
df=df[df.Borough!="Not assigned"]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [5]:
# gather postal code with many neighbourhoods

# First a function to gather
def gather(a):
    if len(a)==1:
        return a
    return ", ".join(a)


dfg=df.groupby(['Postcode','Borough'])
newDf=dfg.agg(gather)
# put new index
newDf.reset_index(inplace=True)
newDf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
#Taking care of empty neighbourhoods
for a in newDf.index:
    if newDf.loc[a][2]=='':
        newDf.loc[a][2]=newDf.loc[a][1]


newDf

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [7]:
newDf.shape


(103, 3)

## Part 2: Getting geographical coordinates

In [8]:
gpd=pd.read_csv('Geospatial_Coordinates.csv')
gpd.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
finalDf=pd.concat([newDf,gpd[['Latitude','Longitude']]],axis=1,sort=False)
finalDf

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
